# Intro to Deep Learning
<table align="left"><td>
  <a target="_blank"  href="https://github.com/Clemson-AI/Intro/blob/master/Intro_to_Deep_Learning_CAI.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on github
  </a>
</td><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/Clemson-AI/Intro/blob/master/Intro_to_Deep_Learning_CAI.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
</td></table>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
from google.colab import files

# Pytorch Tensors
<img src="https://miro.medium.com/max/1050/0*jGB1CGQ9HdeUwlgB" width=70% height=70% alt="Tensor">

In [ ]:
# Rank 1 
torch.tensor([.7, 1.4, 2.1])

In [ ]:
# Rank 2
torch.randn(2,2)

In [ ]:
# Rank 4
torch.randn(2,2,2,2)

# Matrix Multiplication

In [ ]:
t1 = torch.tensor([1.0, 2.0, 3.0])
t2 = torch.tensor([[.3, .2], [.5, .5], [.2, .2]])
t3 = torch.tensor([])

In [ ]:
result = t1.matmul(t2)

In [ ]:
result

# Defining Activation Functions

\begin{equation}

Sigmoid(x) = \sigma(x) = \dfrac{1}{1 + \exp(-x)}

\end{equation}

![Sigmoid](https://pytorch.org/docs/stable/_images/Sigmoid.png)
\begin{equation}

ReLU(x) = x^+ = max(0,x)
\end{equation}

![ReLu](https://pytorch.org/docs/stable/_images/ReLU.png)



Sigmoid

In [ ]:
# Define non linearity
sigmoid = nn.Sigmoid()
relu = nn.ReLU()

In [ ]:
# Take the sigmoid of our example
test = sigmoid(result).matmul(torch.tensor([.4, .7]))

In [ ]:
sigmoid(test)

ReLU

In [ ]:
# Rectified Linear Unit (ReLU)
relu(result)

# Loss

### <center>L1Loss</center>
\begin{equation}
loss = \dfrac{\sum_{i=1}^{n}∣y_i−\hat{y}_i∣}{n}
\end{equation}

### <center>Binary Cross Entropy Loss</center>
<p align="center">
  <img src="https://miro.medium.com/max/1096/1*rdBw0E-My8Gu3f_BOB6GMA.png" />
</p>

### Regression

In [ ]:
data = torch.tensor([5.0,5.0,5.0])
truth = torch.tensor([7.0,8.0,9.0])

In [ ]:
# L1 Loss, mean absolute error (MAE) useful for Regression tasks
criterion = nn.L1Loss(reduction='mean')

In [ ]:
criterion(data, truth)

### Classification

In [ ]:
data = torch.tensor([.34, .25, 1.0])
truth = torch.tensor([1.0, 0.0, 1.0])

In [ ]:
# BCELoss is useful for Classification tasks
criterion = nn.BCELoss()

In [ ]:
criterion(data, truth)

# Linear Layers

In [ ]:
# Remember our inputs and weight
print("input: {}\nweight: {}".format(t1, t2))

In [ ]:
# Perceptrons are called Linear Layers in Pytorch. 
# For example, this will take 3 inputs and output 2
example = nn.Linear(3, 2, bias=False)

# Setting weights as in presentation
example.weight = nn.Parameter(t2, requires_grad=False)

In [ ]:
example.weight

In [ ]:
# Forward Pass
example(t1)

# Define A Neural Network
To define a network in Pytorch, we can extend nn.Module. To use functions without defining them, we can use nn.functional

In [ ]:
import torch.nn.functional as F
IMG_SIZE = 150528
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # Define your layers here
        self.fc1 = nn.Linear(in_features=IMG_SIZE, out_features=, bias=)
        self.fc2 = nn.Linear(1, 1) # bias defaults to True if you do not set it
        #define more here

        #define dropout
        self.dropout = nn.Dropout(p=.5)

    def forward(self, x):
        ### Do not change this
        x = x.view(-1, 3*224*224)
        ###
        x = self.fc1(x) # Pass through first layer
        x = F.relu(x) # Apply non-linearity
        x = self.dropout(x)
        x = F.relu(self.fc2(x)) # you can also combine them like this
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return torch.sigmoid(x).squeeze() # Use sigmoid on final output, we want our results 0-1 (cat to dog)

In [ ]:
# Create an instance of your defined model
my_model = Model()

# Optimizer
To define an optimizer we have to import torch.optim:
https://pytorch.org/docs/stable/optim.html  
* [Adam: A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980)
* [An overview of gradient descent optimization algorithms∗](https://arxiv.org/pdf/1609.04747.pdf)

In [ ]:
import torch.optim as optim
# Define your Optimizer or choose one, also choose a learning rate 
#optimizer = optim.SGD(my_model.parameters(), lr=0.01) 
#optimizer = optim.Adam(Model.parameters(), lr=0.01)

# Use BCELoss for Binary Classification
criterion = nn.BCELoss()

# Dataset
Data processing is important, but we'll implement this part for you.

In [ ]:
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

In [ ]:
!unzip -q kagglecatsanddogs_3367a.zip

In [ ]:
from PIL import Image 
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [ ]:
import os

for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        check = check_image(fpath)

        if not check:
            # Delete corrupted image
            os.remove(fpath)

In [ ]:
len(data.samples)

In [ ]:
from torchvision import datasets
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torch.utils.data import random_split

batch_size = 96

transform = T.Compose([T.RandomHorizontalFlip(),
                       T.Resize((224,224)),
                       T.ToTensor(),
                       T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                       ])


data = datasets.ImageFolder("PetImages", transform=transform)

train_data, test_data = random_split(data, [20000, 4998])
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# Train

In [ ]:
use_cuda = torch.cuda.is_available()
if use_cuda:
  my_model.cuda()

In [ ]:
from math import ceil

NUM_BATCH = ceil(19998/96)
NUM_EPOCH = 2

for i in range(NUM_EPOCH):
  my_model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
            # move to GPU
            train_loss = 0.0
            target = target.type(torch.FloatTensor)
            if use_cuda:
               data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()
            output = my_model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            print('Epoch: {} {}/{}\tTraining Loss: {:.6f}'.format(
            i, 
            batch_idx,
            NUM_BATCH,
            train_loss
            ))

# Test Accuracy

In [ ]:
from math import ceil
t_model.eval()
NUM_BATCH = ceil(4998/96)

correct = 0
total = 0
for batch_idx, (data, target) in enumerate(test_loader):
      # move to GPU
      target = target.type(torch.FloatTensor)
      if use_cuda:
          my_model.cuda()
          vgg16.cuda()
          data, target = data.cuda(), target.cuda()
      output = my_model(data)
      total += len(output)
      for ypred, y in zip(output, target):
        if round(ypred.item()) == round(y.item()): 
          correct += 1
          

      print('{}/{}\t Correct: {}/{}'.format(
        batch_idx,
        NUM_BATCH,
        correct,
        total
        ))
print("Accuracy: {:.2%}".format(correct/total))

Well done on training a MLP!  
I got an accuracy of **60.86%** lmk if you beat this!

# Saving and Loading
If you like your model, save it!

In [ ]:
torch.save(my_model, "my_model.pt")

In [ ]:
# You can attach your Google Drive to copy the saved model
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp my_model.pt /content/drive/MyDrive/

In [ ]:
model = torch.load("my_model.pt")

# Transfer Learning with VGG Backbone
For a more powerful model, we can use a CNN backbone. VGG-16 is a CNN. CNNs are commonly used to extract features from images. Pytorch lets us import some pretrained models from torchvision.models.  

* [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)

In [ ]:
import torchvision.models as models
# vgg16 is a Convolutional Neural Network(CNN) trained on Imagenet 2014 - 1000 categories and 1.3 million images
vgg16 = models.vgg16(pretrained=True)

# Turn off training for vgg16
for param in vgg16.parameters():
    param.requires_grad = False

In [ ]:
# We are going to reimplement the classifier layer for our task
# For reference, this is the original
vgg16.classifier

# Transfer Learning Explained
Using a pretrained VGG backbone will dramatically increase our model accuracy. If you are curious about why transfer learning works, I've attached a video by Andrew Ng. He is the founder of coursera and [deeplearning.ai](https://deeplearning.ai). You can check them out for more content and subscribe to his weekly AI newsletter, The Batch.

In [ ]:
# Embed youtube video 
from IPython.display import YouTubeVideo, display
video = YouTubeVideo("yofjFQddwHE", width=500)
display(video)

# Defining a Classification Layer

In [ ]:
VGG_OUT_FEATURES = vgg16.classifier[0].in_features

In [ ]:
import torch.nn.functional as F
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # Define your layers here
        self.fc1 = nn.Linear(in_features=VGG_OUT_FEATURES, out_features=)
        self.fc2 = nn.Linear(1 ,1)

    def forward(self, x):
        ### Do not change this section
        x = vgg16.features(x)
        x = vgg16.avgpool(x)
        x = x.view(-1, 7*7*512)
        ###

        x = F.relu(self.fc1(x))
        x = 

        return torch.sigmoid(x).squeeze()

In [ ]:
t_model = Model()

# Train

In [ ]:
use_cuda = torch.cuda.is_available()
if use_cuda:
  t_model.cuda()
  vgg16.cuda()

In [ ]:
import torch.optim as optim
criterion = nn.BCELoss()
optimizer = optim.Adam(t_model.parameters(), lr=0.001)
NUM_EPOCH = 2

In [ ]:
from math import ceil
NUM_BATCH = ceil(19998/96)
for i in range(NUM_EPOCH):
  t_model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
            # move to GPU
            train_loss = 0.0
            target = target.type(torch.FloatTensor)
            if use_cuda:
               data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()
            output = t_model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            print('Epoch: {} {}/{}\tTraining Loss: {:.6f}'.format(
            i, 
            batch_idx,
            NUM_BATCH,
            train_loss
            ))

# Inference

In [ ]:
# Let's see how it does on the test dataset
from math import ceil
t_model.eval()
NUM_BATCH = ceil(4998/96)

correct = 0
total = 0
for batch_idx, (data, target) in enumerate(test_loader):
      # move to GPU
      target = target.type(torch.FloatTensor)
      if use_cuda:
          t_model.cuda()
          vgg16.cuda()
          data, target = data.cuda(), target.cuda()
      output = t_model(data)
      total += len(output)
      for ypred, y in zip(output, target):
        if round(ypred.item()) == round(y.item()): 
          correct += 1
          

      print('{}/{}\t Correct: {}/{}'.format(
        batch_idx,
        NUM_BATCH,
        correct,
        total
        ))
print("Accuracy: {:.2%}".format(correct/total))

In [ ]:
# Don't need gpu for one image
t_model.cpu()
vgg16.cpu()

In [ ]:
from PIL import Image 
def process(im_path):  
  im = Image.open(im_path)
  return transform(im)[:3,:,:].unsqueeze(0)

In [ ]:
# Upload your own image
from google.colab import files
files.upload()

In [ ]:
# Process Image
img = process("download.jpg")

In [ ]:
# Cats were labeled 0 and Dogs 1
key = {
    0 : "Cat",
    1 : "Dog"
}
prediction = t_model(im)

# Show your image
im = cv2.imread("download.jpg")
cv2_imshow(im)

print(key[round(prediction.item())], "{:.2%}".format(abs((prediction.item()-.5)*2)))